In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime

import pandas as pd
import numpy as np
import boto3

import mtglobals

In [3]:
client, mturk_environment = mtglobals.gen_client()
all_hits = mtglobals.download_all_hits(client, start_cutoff=datetime.datetime(2021,12,1))

Your account balance is 498226.73
p2:DyJwfnckJYls30yXCW5Dru9/FQNDSqMSgPA5u59b9BpSbEvPihxQXrSvi6t/21I=
p2:4MKsvrGCvEyRt/WhFBuGIiHPDumxFaL2a5ofgdyMbkf7UHbysusiDHOauBtR3js=
p2:A4FRbnXaXPlFYrltKt+OAErJk8Bcwf53DmQDYJlLgc0mYg50Et/QciCDo1h/j0o=
p2:QqkITXkf6YzZhM+m6vX4dLidBWxFgNzrujoaobG84GrLa2cqfb7xFpI9zHLgAmE=
p2:X4JkcjjC9gYOaF4IsMysrKyiO1yJfuSA/Hy8O3AXsmo9mT0+MLFkSoMW77AHYOs=
p2:7q9GyJd2yczZ929giNI/AhlBb0rrCfpdzYuDYKm6wwhLmpAQtFpPibH3e7Vg3eI=
p2:0u2JWN0aFyT6yYpPDkhsLQxuOOMmfqoKbZgtQw7nJm7RSBbSLbcoMUO8GyUqyA8=
p2:1KZcHGhiNjYmLSHJ9JXyMGMtKHB7LyHf6hY/Wp6xGpL4hVk8dRVRGg+kpFU3d6E=
p2:twiR4f4sajDT3gTGQT0JmeyB0H+JKDHPG47b166ZgHvLWtD4wPteYNmMPcqbsTwE
p2:1YrjdNHGaFKXbaN7V3/nK6qlLXqeOut7k8Fh6kjBKXs8UH+46swVLAkrxWooTCxF
p2:mzQodmkrSjRbiorVekjnH86PDjEMsPuQjBbI7WCuxDGG2bCK1hRrLsTjg8P6hg7z
p2:Yu02WTFlgtDsXxqJCrYMZ3sNC+trdutvjgj4v7W4Od4zVLEZo6+KQlSF2BzNyWv4
p2:APjs6QwF1R/lQ6RJTMe8f+ZY//vmip37vxl9N6szGM8xD/jFN3FehDisTaNFxIK6
p2:bPDinM9Cb3N7lObxhzySDK04VZiMINSR1zhan7xJsWmZhd0RSnXUBi6JVNrPg2Qd
p2:92A3swX4SW2

In [4]:
current_qual = mtglobals.get_current_qual()
current_qual['qual_name']

'Workplace_Survey_Custom13'

In [5]:
worker_ids = mtglobals.get_workers_with_qual(client, current_qual['qual_name'])

In [6]:
len(worker_ids)

1000

In [7]:
# Print titles for each HIT
#hit_list = [(r_num, r['Title'], r['NumberOfAssignmentsAvailable']) for r_num, r in enumerate(all_hits)]
#len(hit_list)
#hit_list[:20]

In [8]:
# Now load the launched worker HIT info
#launched_df = pd.read_csv(mtglobals.stage2_launched_fpath)
#len(launched_df)

In [9]:
def get_most_recent(worker_hits):
    sorted_hits = sorted(worker_hits, key=lambda x: x['CreationTime'], reverse=True)
    #print([h['CreationTime'] for h in sorted_hits])
    return sorted_hits[0]

def get_active_hits(all_hits, launched_df):
    active_hits = []
    for row_index, cur_row in launched_df.iterrows():
        cur_worker_id = cur_row['worker_id']
        cur_launch = cur_row['launched_time']
        #print(f"{cur_worker_id}: {cur_launch}")
        # Find the HIT for this worker
        worker_hits = [h for h in all_hits if cur_worker_id in h['Title']]
        if len(worker_hits) > 1:
            # Multiple HITs for this worker...
            print(f"{len(worker_hits)} HITs for {cur_worker_id}")
            # Use CreationTime to find the most recent HIT
            #for hit_num, cur_hit in enumerate(worker_hits):
            #    print(f"HIT #{hit_num}: {cur_hit['CreationTime']}")
            most_recent = get_most_recent(worker_hits)
            #print(f"Most recent: {most_recent['CreationTime']}")
            #print(worker_hits)
            active_hits.append((cur_worker_id,most_recent))
        elif len(worker_hits) == 1:
            #print(f"Found unique HIT for {cur_worker_id}")
            active_hits.append((cur_worker_id, worker_hits[0]))
        else:
            print(f"No HIT found for {cur_worker_id}!")
    return active_hits

In [10]:
def get_hit_for_worker(all_hits, worker_id, verbose=False):
    vprint = print if verbose else lambda x: None
    # The HIT only has the worker_id in its title, so we have to extract
    # using mtglobals.worker_id_from_title()
    #vprint([h['Title'] for h in all_hits])
    results = [h for h in all_hits if mtglobals.worker_id_from_title(h['Title']) == worker_id]
    if len(results) == 0:
        raise Exception(f"No HIT found for worker {worker_id}")
    return results[0]

In [11]:
# Get the hits created for all workers with the qual specified above
stage2_hits = [get_hit_for_worker(all_hits, w, verbose=True) for w in worker_ids]

In [12]:
#stage2_hits = [h for h in all_hits if h['Title'].startswith("Custom workplace survey HIT")]
#stage2_hit_info = get_active_hits(all_hits, launched_df)
#stage2_hits = [i[1] for i in stage2_hit_info]
#print(f"{len(stage2_hits)} stage-2 HITs found")
#stage2_ids = [h['HITId'] for h in stage2_hits]
#info_vars = ['HITId','HITTypeId','HITGroupId','HITLayoutId', #'Title', 'Description',
#            'Reward']

In [13]:
def check_submitted(hit_id):
    # Return 
    assignment_response = client.list_assignments_for_hit(
        HITId=hit_id,
        #AssignmentStatuses=[
        #    'Submitted'|'Approved'|'Rejected',
        #]
    )
    return assignment_response

In [14]:
[
    (n, str(h['CreationTime']), h['Title'], h['Reward'], h['NumberOfAssignmentsCompleted'],
     h['NumberOfAssignmentsPending'], h['NumberOfAssignmentsAvailable'])
    for n, h in enumerate(stage2_hits)
];

In [15]:
def process_assignments(assignment_response):
    num_results = assignment_response['NumResults']
    assignments = assignment_response['Assignments']
    if num_results > 0:
        #print(f"=====[ Submitted HIT: {cur_title} ]=====")
        # Approve Stage2 submission
        cur_assignment = assignments[0]
        submitted_list.append(cur_assignment)
        assignment_id = cur_assignment['AssignmentId']
        status = cur_assignment['AssignmentStatus']
        if status == "Submitted":
            print(f"Status: {status}")
            response = client.approve_assignment(
                AssignmentId=assignment_id,
                OverrideRejection=True
            )
            print(f"Approved assignment {assignment_id}")
            print(response)


approve_submitted = True
submitted_list = []
all_submit_info = []
for hit_num, cur_hit in enumerate(stage2_hits):
    cur_title = cur_hit['Title']
    cur_creation = cur_hit['CreationTime']
    #print("=====[ start loop iteration ]=====")
    print(f"HIT #{hit_num}: {cur_title}, created {cur_creation}")
    #print()
    cur_id = cur_hit['HITId']
    cur_reward = cur_hit['Reward']
    num_avail = cur_hit['NumberOfAssignmentsAvailable']
    assignment_response = check_submitted(cur_id)
    if approve_submitted:
        process_assignments(assignment_response)
    # Get submission info
    cur_worker_id = mtglobals.worker_id_from_title(cur_title)
    submit_info = (cur_worker_id, cur_reward, num_avail)
    all_submit_info.append(submit_info)
    #print("----- end loop iteration -----\n")

HIT #0: Custom workplace survey HIT for worker id A002160837SWJFPIAI7L7, created 2022-01-21 13:08:53-08:00
HIT #1: Custom workplace survey HIT for worker id A0803926V8JYOTXBYT4T, created 2022-01-21 13:06:15-08:00
Status: Submitted
Approved assignment 3UOUJI6MTEF3JJV0X1ETU0MFHQMUXW
{'ResponseMetadata': {'RequestId': '84fe2b80-7d12-4d06-8b65-b479d69fa8a8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '84fe2b80-7d12-4d06-8b65-b479d69fa8a8', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sun, 23 Jan 2022 21:13:46 GMT'}, 'RetryAttempts': 0}}
HIT #2: Custom workplace survey HIT for worker id A101TTS7WAVXAV, created 2022-01-21 14:45:34-08:00
Status: Submitted
Approved assignment 3BGYGHDBBYLTYRJ1QSJUYRXRLRC22N
{'ResponseMetadata': {'RequestId': '95e3de75-54ed-4117-a2f6-36ff27e9753b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '95e3de75-54ed-4117-a2f6-36ff27e9753b', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 

In [16]:
len(all_submit_info)

1000

In [17]:
all_submit_info[0]

('A002160837SWJFPIAI7L7', '0.90', 1)

In [18]:
# Just switches 0 and 1 so it's 1 for accepted and 0 otherwise
accepted_info = [(i[0], i[1], 1-i[2]) for i in all_submit_info]

Select only the expired HITs here, if necessary

In [19]:
#accepted_info = accepted_info[:8]

In [20]:
# Transform this into a .dta for regression
accepted_df = pd.DataFrame(accepted_info, columns=['worker_id','wage','accepted'])

In [21]:
accepted_df['lwage'] = accepted_df['wage'].apply(lambda x: np.log(float(x)))

In [22]:
#accepted_df.to_stata('../results_2stage/accepted_reg_df.dta')

In [23]:
accepted_df

,worker_id,wage,accepted,lwage
0,A002160837SWJFPIAI7L7,0.90,0,-0.105361
1,A0803926V8JYOTXBYT4T,1.05,1,0.048790
2,A101TTS7WAVXAV,0.95,1,-0.051293
3,A105ERW1E8SDP5,1.01,1,0.009950
4,A10I1CHTHMN5EZ,0.90,0,-0.105361
...,...,...,...,...
995,AZ5QXTWJVP5T9,1.05,1,0.048790
996,AZ7G576MJ5HEB,1.50,1,0.405465
997,AZQDLP9JE0SFB,1.10,0,0.095310
998,AZU9D7YLCTAYV,1.02,1,0.019803


In [24]:
# Get just the info for the most recent HITs
#accepted_info_sub = accepted_info[-106:]
#accepted_info_sub;

In [25]:
totals = {'overall':0}
accepts = {'overall':0}
for cur_info in accepted_info:
    cur_wage = cur_info[1]
    if cur_wage not in totals:
        totals[cur_wage] = 0
    if cur_wage not in accepts:
        accepts[cur_wage] = 0
    cur_accepted = cur_info[2]
    totals['overall'] = totals['overall'] + 1
    totals[cur_wage] = totals[cur_wage] + 1
    if cur_accepted == 1:
        accepts['overall'] = accepts['overall'] + 1
        accepts[cur_wage] = accepts[cur_wage] + 1

In [26]:
totals

{'overall': 1000,
 '0.90': 82,
 '1.05': 88,
 '0.95': 109,
 '1.01': 76,
 '1.02': 101,
 '0.50': 92,
 '1.50': 101,
 '1.10': 110,
 '0.99': 78,
 '1.00': 83,
 '0.98': 80}

In [27]:
accepts

{'overall': 735,
 '0.90': 56,
 '1.05': 67,
 '0.95': 82,
 '1.01': 60,
 '1.02': 74,
 '0.50': 61,
 '1.50': 76,
 '1.10': 82,
 '0.99': 58,
 '1.00': 62,
 '0.98': 57}

In [28]:
import statsmodels.formula.api as smf
result = smf.ols(formula='accepted ~ lwage', data=accepted_df).fit(cov_type='HC1')
intercept = result.params[1]

In [29]:
print(result.params)

Intercept    0.737074
lwage        0.094659
dtype: float64


In [30]:
for cur_wage in sorted(totals.keys()):
    header = f"{cur_wage}:" if cur_wage == "overall" else f"${cur_wage}:"
    print(header)
    print(f"{accepts[cur_wage]} accepted out of {totals[cur_wage]} offers")
    accept_rate = accepts[cur_wage] / totals[cur_wage]
    print(f"Accept rate: {accept_rate}")
print(f"Regression intercept: {intercept:.4f}")

$0.50:
61 accepted out of 92 offers
Accept rate: 0.6630434782608695
$0.90:
56 accepted out of 82 offers
Accept rate: 0.6829268292682927
$0.95:
82 accepted out of 109 offers
Accept rate: 0.7522935779816514
$0.98:
57 accepted out of 80 offers
Accept rate: 0.7125
$0.99:
58 accepted out of 78 offers
Accept rate: 0.7435897435897436
$1.00:
62 accepted out of 83 offers
Accept rate: 0.7469879518072289
$1.01:
60 accepted out of 76 offers
Accept rate: 0.7894736842105263
$1.02:
74 accepted out of 101 offers
Accept rate: 0.7326732673267327
$1.05:
67 accepted out of 88 offers
Accept rate: 0.7613636363636364
$1.10:
82 accepted out of 110 offers
Accept rate: 0.7454545454545455
$1.50:
76 accepted out of 101 offers
Accept rate: 0.7524752475247525
overall:
735 accepted out of 1000 offers
Accept rate: 0.735
Regression intercept: 0.0947


And now compute the elasticity

In [31]:
result = smf.ols(formula='accepted ~ lwage', data=accepted_df).fit(cov_type='HC1')
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               accepted   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2.658
Date:                Sun, 23 Jan 2022   Prob (F-statistic):              0.103
Time:                        13:19:11   Log-Likelihood:                -599.54
No. Observations:                1000   AIC:                             1203.
Df Residuals:                     998   BIC:                             1213.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7371      0.014     52.852      0.0

### Save list of submitters

In [35]:
# Save list of submitters
submitter_list = []
for cur_assignment in submitted_list:
    worker_id = cur_assignment['WorkerId']
    submitter_list.append(worker_id)

In [36]:
submit_df = pd.DataFrame({'worker_id':submitter_list})

In [37]:
csv_fpath = mtglobals.stage2_submitted_fpath
submit_df.to_csv(csv_fpath, index=False)

In [38]:
# But also save a df with *all* the launched stage2 HITs, including creation times
launched_data = []
for cur_hit in stage2_hits:
    cur_id = cur_hit['HITId']
    cur_title = cur_hit['Title']
    cur_reward = cur_hit['Reward']
    cur_creation = cur_hit['CreationTime']
    num_completed = cur_hit['NumberOfAssignmentsCompleted']
    cur_data = {'hit_id':cur_id, 'title':cur_title, 'reward':cur_reward,
                'creation':cur_creation, 'completed':num_completed}
    launched_data.append(cur_data)

In [39]:
launched_df = pd.DataFrame(launched_data)

In [40]:
import pytz
pacific = pytz.timezone('US/Pacific')
one_day_ago = datetime.datetime.now(pacific) - datetime.timedelta(days=1)
one_day_ago

datetime.datetime(2022, 1, 15, 15, 21, 41, 139420, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [41]:
launched_df.iloc[0]['creation'] < one_day_ago

True

In [42]:
launched_df['expired'] = launched_df['creation'] < one_day_ago

In [43]:
len(launched_df[launched_df['expired']])

500

In [44]:
expired_df = launched_df[launched_df['expired']].copy()

In [45]:
expired_df['reward']

0      0.90
1      0.99
2      0.95
3      1.10
4      1.02
       ... 
495    0.99
496    1.50
497    1.50
498    1.02
499    0.98
Name: reward, Length: 500, dtype: object

In [46]:
expired_df = expired_df[expired_df['reward'].isin(["0.50","1.50"])].copy()

In [47]:
expired_df

,hit_id,title,reward,creation,completed,expired
8,3X4Q1O9UBINB4F7IT0TKDBLBQW1O7B,Custom workplace survey HIT for worker id A147...,1.50,2022-01-13 10:12:16-08:00,1,True
9,3L7SUC0TTVV9M1MMVLTY6SMJNIH0M9,Custom workplace survey HIT for worker id A153...,1.50,2022-01-13 10:09:05-08:00,1,True
20,3JU8CV4BRMD0ALBIPMN1DXCWQQ7POJ,Custom workplace survey HIT for worker id A17V...,1.50,2022-01-13 10:06:48-08:00,1,True
35,3E9ZFLPWOZTVN826DIKDND2DGGHIX5,Custom workplace survey HIT for worker id A1BL...,1.50,2022-01-13 10:09:15-08:00,1,True
40,3AQN9REUTGH3F2CUQ054FUT9QH3DY4,Custom workplace survey HIT for worker id A1D8...,0.50,2022-01-13 10:06:33-08:00,0,True
...,...,...,...,...,...,...
478,3JYPJ2TAYJ9648YPQJ4BQ33IYPGPFX,Custom workplace survey HIT for worker id AT3G...,1.50,2022-01-13 10:05:13-08:00,1,True
488,3IJ95K7NDYDCFNI7LOX14ZYZZPFNGY,Custom workplace survey HIT for worker id AWB5...,1.50,2022-01-13 10:08:42-08:00,0,True
489,3GL25Y6844VHJRSXYXI5F69UXP1XM0,Custom workplace survey HIT for worker id AWD7...,1.50,2022-01-13 10:07:05-08:00,1,True
496,39RRBHZ0AV2PE9MLD77HZN8IWIBVZC,Custom workplace survey HIT for worker id AYT6...,1.50,2022-01-13 10:11:23-08:00,1,True


In [41]:
low_df = expired_df[expired_df['reward'] == "0.50"].copy()
high_df = expired_df[expired_df['reward'] == "1.50"].copy()